In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("order_transactions.csv")

In [3]:
data.head()

,CUSTNAME,GenderCode,ADDRESS1,CITY,STATE,COUNTRY_CODE,POSTAL_CODE,POSTAL_CODE_PLUS4,ADDRESS2,EMAIL_ADDRESS,...,Cleaning Products,Condiments,Frozen Foods,Kitchen Items,Meat,Office Supplies,Personal Care,Pet Supplies,Sea Food,Spices
0,Allen Perl,Mr.,4707 Hillcrest Lane,Abeto,PG,IT,6040,0,NaN,Allen.M.Perl@spambob.com,...,0,1,0,0,0,0,0,0,0,0
1,Allen Perl,Mr.,4707 Hillcrest Lane,Abeto,PG,IT,6040,0,NaN,Allen.M.Perl@spambob.com,...,1,0,0,0,0,0,0,0,0,0
2,Allen Perl,Mr.,4707 Hillcrest Lane,Abeto,PG,IT,6040,0,NaN,Allen.M.Perl@spambob.com,...,0,0,0,0,0,0,0,0,0,0
3,Allen Perl,Mr.,4707 Hillcrest Lane,Abeto,PG,IT,6040,0,NaN,Allen.M.Perl@spambob.com,...,0,0,0,0,0,0,0,0,0,0
4,Allen Perl,Mr.,4707 Hillcrest Lane,Abeto,PG,IT,6040,0,NaN,Allen.M.Perl@spambob.com,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_filtered = data[['CUST_ID','Baby Food', 'Diapers', 'Formula', 'Lotion', 'Baby wash', 'Wipes', 'Fresh Fruits', 'Fresh Vegetables', 'Beer', 'Wine', 'Club Soda', 'Sports Drink', 'Chips', 'Popcorn', 'Oatmeal', 'Medicines', 'Canned Foods', 'Cigarettes', 'Cheese', 'Cleaning Products', 'Condiments', 'Frozen Foods', 'Kitchen Items', 'Meat', 'Office Supplies', 'Personal Care', 'Pet Supplies', 'Sea Food', 'Spices']]

In [5]:
df_customer_products = df_filtered.groupby('CUST_ID').sum()

In [6]:
df_customer_products.head()

,Baby Food,Diapers,Formula,Lotion,Baby wash,Wipes,Fresh Fruits,Fresh Vegetables,Beer,Wine,...,Cleaning Products,Condiments,Frozen Foods,Kitchen Items,Meat,Office Supplies,Personal Care,Pet Supplies,Sea Food,Spices
CUST_ID,,,,,,,,,,,,,,,,,,,,,
10001,0,0,1,0,0,1,0,1,0,1,...,1,0,0,0,0,0,0,0,1,1
10002,0,0,0,0,0,0,0,0,0,2,...,0,0,2,0,0,0,3,0,2,1
10003,0,1,2,2,1,0,0,1,1,0,...,2,1,0,0,0,0,0,0,0,0
10004,0,1,0,0,0,0,2,0,1,3,...,1,0,2,0,2,0,2,0,2,3
10005,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 0:
        return 1
    
df_basket = df_customer_products.applymap(encode_units)

In [9]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [10]:
my_frequent_itemsets = apriori(df_basket, min_support=0.07, use_colnames=True)
my_frequent_itemsets.head() 

,support,itemsets
0,0.140463,(Baby Food)
1,0.247613,(Diapers)
2,0.147889,(Formula)
3,0.133885,(Lotion)
4,0.141736,(Baby wash)


In [11]:
my_rules = association_rules(my_frequent_itemsets, metric="lift", min_threshold=1)
my_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Wine),(Baby Food),0.361341,0.140463,0.083174,0.230182,1.638743,0.032419,1.116546
1,(Baby Food),(Wine),0.140463,0.361341,0.083174,0.592145,1.638743,0.032419,1.565897
2,(Baby Food),(Cigarettes),0.140463,0.364099,0.084023,0.598187,1.642924,0.032881,1.582580
3,(Cigarettes),(Baby Food),0.364099,0.140463,0.084023,0.230769,1.642924,0.032881,1.117399
4,(Condiments),(Baby Food),0.543603,0.140463,0.077658,0.142857,1.017048,0.001302,1.002794


In [12]:
my_rules[(my_rules['lift'] >= 3) & (my_rules['confidence'] >= 0.5)].sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(Beer),(Diapers),0.146404,0.247613,0.146404,1.000000,4.038560,0.110152,inf
246,"(Beer, Cigarettes)",(Diapers),0.088479,0.247613,0.088479,1.000000,4.038560,0.066570,inf
4278,"(Condiments, Meat)","(Spices, Cigarettes)",0.081052,0.298748,0.081052,1.000000,3.347301,0.056838,inf
258,"(Spices, Beer)",(Diapers),0.086145,0.247613,0.086145,1.000000,4.038560,0.064814,inf
1210,"(Canned Foods, Pet Supplies)",(Cleaning Products),0.074687,0.245703,0.074687,1.000000,4.069948,0.056336,inf
...,...,...,...,...,...,...,...,...,...
8122,"(Sea Food, Personal Care)","(Spices, Wine, Cigarettes, Frozen Foods)",0.197751,0.154679,0.099512,0.503219,3.253320,0.068924,1.701597
7602,"(Cheese, Wine, Frozen Foods)","(Spices, Medicines, Personal Care)",0.173138,0.154042,0.086993,0.502451,3.261779,0.060323,1.700251
9195,"(Spices, Wine, Cigarettes, Medicines)","(Sea Food, Personal Care, Frozen Foods)",0.156588,0.144918,0.078294,0.500000,3.450220,0.055602,1.710163
9080,"(Spices, Cheese, Personal Care)","(Wine, Sea Food, Cigarettes, Medicines)",0.155739,0.152132,0.077870,0.500000,3.286611,0.054177,1.695735


In [14]:
temp = my_rules[(my_rules['lift'] >= 3) & (my_rules['confidence'] >= 0.5)].sort_values(by=['support','confidence','lift'], ascending=False)

In [38]:
temp.iloc[1374]

antecedents           (Cheese, Sea Food, Spices, Medicines, Personal...
consequents                                        (Wine, Frozen Foods)
antecedent support                                             0.090813
consequent support                                             0.245916
support                                                        0.075111
confidence                                                     0.827103
lift                                                           3.363361
leverage                                                       0.052779
conviction                                                     4.361461
Name: 9284, dtype: object